In [37]:
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# ax1.spines['top'].set_visible(False) 

import os
# import csv
import math

from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]

saveFigures is set to: True
Done loading packages


In [38]:
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"

path_figs = os.getcwd() +"\\..\\Figures\\" 

In [39]:
latestsubdir = list(os.walk(path_data))[0][1][-1]
latestdir = path_data + latestsubdir

dfCase = pd.read_csv(latestdir+'/Test_pos_over_time.csv',delimiter = ';',dtype=str)
dfCase = dfCase.iloc[:-2]
dfCase['NewPositive'] = pd.to_numeric(dfCase['NewPositive'].astype(str).apply(lambda x: x.replace('.','')))
dfCase['Tested'] = pd.to_numeric(dfCase['Tested'].astype(str).apply(lambda x: x.replace('.','')))
dfCase['PosPct'] = pd.to_numeric(dfCase['PosPct'].astype(str).apply(lambda x: x.replace(',','.')))
dfCase['Date'] =  pd.to_datetime(dfCase.Date,format='%Y-%m-%d')
testDates = dfCase['Date']

dfAdm = pd.read_csv(latestdir+'/Newly_admitted_over_time.csv',delimiter = ';',dtype=str)
dfAdm['Dato'] = pd.to_datetime(dfAdm['Dato'])
dfAdm['Total'] = pd.to_numeric(dfAdm['Total'])
dfAdm.tail()


dfDea = pd.read_csv(latestdir+'/Deaths_over_time.csv',delimiter = ';',dtype=str)
dfDea = dfDea.iloc[:-1,:]
dfDea['Dato'] = pd.to_datetime(dfDea['Dato'])
dfDea['Antal_døde'] = pd.to_numeric(dfDea['Antal_døde'])
dfDea.tail()


,Dato,Antal_døde
656,2021-12-27,8
657,2021-12-28,11
658,2021-12-29,7
659,2021-12-30,7
660,2021-12-31,0


In [40]:
allDates = dfCase.Date
allDatesShift = allDates + np.timedelta64(365,'D')

allDatesAdm = dfAdm.Dato
allDatesAdmShift = allDatesAdm + np.timedelta64(365,'D')

allDatesDea = dfDea.Dato
allDatesDeaShift = allDatesDea + np.timedelta64(365,'D')


firstDate = np.datetime64('2021-10-01')-np.timedelta64(1,'D')
lastDate = np.datetime64('2022-03-01')+np.timedelta64(1,'D')

meanWidth = 7

# Cases

In [49]:

allCases = dfCase.NewPositive.values

fig,ax1 = plt.subplots(tight_layout=True)


ax1.plot(allDatesShift[:-1],allCases[:-1],'k.:',markersize=4,linewidth=0.5,label='2020/2021')
ax1.plot(rnTime(allDatesShift[:-1],meanWidth),rnMean(allCases[:-1],meanWidth),'k',label=f'2020/2021, {meanWidth} dages gennemsnit')
ax1.plot(allDates[:-1],allCases[:-1],'b.:',markersize=4,linewidth=0.5,label='2021/2022')
ax1.plot(rnTime(allDates[:-1],meanWidth),rnMean(allCases[:-1],meanWidth),'b',label=f'2021/2022, {meanWidth} dages gennemsnit')


ax1.legend(loc='upper left')
ax1.grid()
ax1.set_ylabel('Antal tilfælde')
ax1.set_ylim(bottom=0)
ax1.set_xlim([firstDate,lastDate])
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%B'))
ax1.spines['top'].set_visible(False) 
ax1.spines['right'].set_visible(False)


if saveFigures:
    plt.savefig(path_figs+'VinterSammenligning_Positive')

maxAvg = np.max(rnMean(allCases,meanWidth))
ax1.set_ylim(top=maxAvg*1.1)

if saveFigures:
    plt.savefig(path_figs+'VinterSammenligning_Positive_zoom')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Indlæggelser

In [46]:

allAdms = dfAdm.Total.values

fig,ax1 = plt.subplots(tight_layout=True)


ax1.plot(allDatesAdmShift,allAdms,'k.:',markersize=4,linewidth=0.5,label='2020/2021')
ax1.plot(rnTime(allDatesAdmShift,meanWidth),rnMean(allAdms,meanWidth),'k',label=f'2020/2021, {meanWidth} dages gennemsnit')
ax1.plot(allDatesAdm,allAdms,'b.:',markersize=4,linewidth=0.5,label='2021/2022')
ax1.plot(rnTime(allDatesAdm,meanWidth),rnMean(allAdms,meanWidth),'b',label=f'2021/2022, {meanWidth} dages gennemsnit')


ax1.legend(loc='upper left')
ax1.grid()
ax1.set_ylabel('Antal nyindlæggelser')
ax1.set_ylim(bottom=0)
ax1.set_xlim([firstDate,lastDate])
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%B'))
ax1.spines['top'].set_visible(False) 
ax1.spines['right'].set_visible(False)

if saveFigures:
    plt.savefig(path_figs+'VinterSammenligning_Nyindlagte')

maxAvg = np.max(rnMean(allAdms,meanWidth))
ax1.set_ylim(top=maxAvg*1.1)

if saveFigures:
    plt.savefig(path_figs+'VinterSammenligning_Nyindlagte_zoom')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Dødsfald

In [48]:

allDeas = dfDea['Antal_døde'].values

fig,ax1 = plt.subplots(tight_layout=True)


ax1.plot(allDatesDeaShift[:-1],allDeas[:-1],'k.:',markersize=4,linewidth=0.5,label='2020/2021')
ax1.plot(rnTime(allDatesDeaShift[:-1],meanWidth),rnMean(allDeas[:-1],meanWidth),'k',label=f'2020/2021, {meanWidth} dages gennemsnit')
ax1.plot(allDatesDea[:-1],allDeas[:-1],'b.:',markersize=4,linewidth=0.5,label='2021/2022')
ax1.plot(rnTime(allDatesDea[:-1],meanWidth),rnMean(allDeas[:-1],meanWidth),'b',label=f'2021/2022, {meanWidth} dages gennemsnit')


ax1.legend(loc='upper left')
ax1.grid()
ax1.set_ylabel('Antal dødsfald')
ax1.set_ylim(bottom=0)
ax1.set_xlim([firstDate,lastDate])
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%B'))
ax1.spines['top'].set_visible(False) 
ax1.spines['right'].set_visible(False)

if saveFigures:
    plt.savefig(path_figs+'VinterSammenligning_Doedsfald')

maxAvg = np.max(rnMean(allDeas,meanWidth))
ax1.set_ylim(top=maxAvg*1.1)

if saveFigures:
    plt.savefig(path_figs+'VinterSammenligning_Doedsfald_zoom')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …